In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName("Fuel_Consumption_2000-2022").getOrCreate()

In [3]:
df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("Fuel_Consumption_2000-2022.csv")

In [4]:
df.describe().show()

+-------+------------------+-----+------------------+--------------+------------------+------------------+------------+-----+-----------------+-----------------+------------------+------------------+------------------+
|summary|              YEAR| MAKE|             MODEL| VEHICLE CLASS|       ENGINE SIZE|         CYLINDERS|TRANSMISSION| FUEL| FUEL CONSUMPTION|   HWY (L/100 km)|   COMB (L/100 km)|        COMB (mpg)|         EMISSIONS|
+-------+------------------+-----+------------------+--------------+------------------+------------------+------------+-----+-----------------+-----------------+------------------+------------------+------------------+
|  count|             22556|22556|             22556|         22556|             22556|             22556|       22556|22556|            22556|            22556|             22556|             22556|             22556|
|   mean|2011.5544422769995| NULL| 528.3333333333334|          NULL|3.3566456818584967|5.8541408051072885|        NULL| NULL

In [5]:
class_mapping = {
    'SUV: Small': 'SUV - SMALL',
    'Station wagon: Mid-size': 'STATION WAGON - MID-SIZE',
    'Station wagon: Small': 'STATION WAGON - SMALL',
    'Van: Passenger': 'VAN - PASSENGER',
    'Pickup truck: Small': 'PICKUP TRUCK - SMALL',
    'Pickup truck: Standard': 'PICKUP TRUCK - STANDARD',
    'SUV: Standard': 'SUV - STANDARD'
}

@udf(StringType())
def standardize_vehicle_class(vehicle_class):
    return class_mapping.get(vehicle_class, vehicle_class)

df_clean = df.withColumn('Vehicle Class', standardize_vehicle_class(df['Vehicle Class']))

In [23]:
panda_df = df.toPandas()
panda_df.to_csv("Cleaned_Fuel_Consumption_2000-2022.csv")

In [8]:
# 1. Analyze how fuel efficiency (HWY and COMB) has evolved over time (2000-2022) to identify long-term trends and fluctuations.
filtered_data = df_clean.filter((df_clean['YEAR'] >= 2000) & (df_clean['YEAR'] <= 2022))
fuel_efficiency_by_year = filtered_data.groupBy('YEAR').agg(
    round(avg('HWY (L/100 km)'), 2).alias('avg_HWY_efficiency'),
    round(avg('COMB (L/100 km)'), 2).alias('avg_COMB_efficiency')
)
fuel_efficiency_by_year = fuel_efficiency_by_year.orderBy('YEAR')
fuel_efficiency_by_year.show(fuel_efficiency_by_year.count(), truncate=False)

pandas_df_year = fuel_efficiency_by_year.toPandas()
local_path = "Goal1_fuel_efficiency_by_year.csv"
pandas_df_year.to_csv(local_path, index=False)

+----+------------------+-------------------+
|YEAR|avg_HWY_efficiency|avg_COMB_efficiency|
+----+------------------+-------------------+
|2000|9.07              |11.3               |
|2001|8.99              |11.2               |
|2002|9.32              |11.6               |
|2003|9.22              |11.47              |
|2004|9.26              |11.55              |
|2005|9.1               |11.41              |
|2006|8.98              |11.28              |
|2007|9.06              |11.42              |
|2008|8.92              |11.26              |
|2009|8.9               |11.25              |
|2010|8.67              |10.92              |
|2011|8.67              |10.92              |
|2012|8.51              |10.74              |
|2013|8.23              |10.38              |
|2014|8.01              |10.09              |
|2015|8.93              |10.98              |
|2016|8.84              |10.81              |
|2017|8.98              |10.87              |
|2018|8.99              |10.85    

In [9]:
# 2. Investigate how fuel efficiency varies among different vehicle makes and models to pinpoint high-performing and underperforming vehicles.
fuel_efficiency_by_make_model = df_clean.groupBy('MAKE', 'MODEL').agg(
    round(avg('COMB (mpg)'), 2).alias('avg_COMB_mpg')
)
top_high_performing_vehicles = fuel_efficiency_by_make_model.orderBy('avg_COMB_mpg', ascending=False).limit(10)
top_low_performing_vehicles = fuel_efficiency_by_make_model.orderBy('avg_COMB_mpg').limit(10)
top_high_performing_vehicles.show()
top_low_performing_vehicles.show()

pandas_top_high_performing_vehicles = top_high_performing_vehicles.toPandas()
local_path = "Goal2_top_high_performing_vehicles.csv"
pandas_top_high_performing_vehicles.to_csv(local_path, index=False)
pandas_top_low_performing_vehicles = top_low_performing_vehicles.toPandas()
local_path = "Goal2_top_low_performing_vehicles.csv"
pandas_top_low_performing_vehicles.to_csv(local_path, index=False)

+----------+--------------------+------------+
|      MAKE|               MODEL|avg_COMB_mpg|
+----------+--------------------+------------+
|     HONDA|             INSIGHT|       74.11|
|   Hyundai|          IONIQ Blue|        69.8|
|   HYUNDAI|          IONIQ BLUE|        69.0|
|    TOYOTA|               PRIUS|       67.82|
|     SMART|          FORTWO CDI|        67.0|
|     SMART|FORTWO CDI CABRIOLET|        67.0|
|    TOYOTA|             PRIUS c|       66.67|
|   Hyundai|               IONIQ|        66.6|
|   Hyundai| Elantra Hybrid Blue|        64.0|
|VOLKSWAGEN|  JETTA TURBO HYBRID|        64.0|
+----------+--------------------+------------+

+-----------+--------------------+------------+
|       MAKE|               MODEL|avg_COMB_mpg|
+-----------+--------------------+------------+
|    Bugatti|    Chiron Pur Sport|        11.0|
|    FERRARI|                ENZO|        11.0|
|    Bugatti|  Chiron Super Sport|        11.0|
|Lamborghini|Aventador Sian Ro...|        12.0|
|    

In [11]:
# 3. Assess how engine size, cylinders, and other specifications impact fuel consumption to optimize vehicle design for efficiency
df_clean = df_clean.withColumn('YEAR', df_clean['YEAR'].cast('int'))
trend_analysis_by_year = df_clean.groupBy('YEAR', 'ENGINE SIZE').agg(
    round(avg('FUEL CONSUMPTION'), 2).alias('avg_fuel_consumption')
)
trend_analysis_by_year.count()

pandas_trend_analysis_by_year = trend_analysis_by_year.toPandas()
local_path = "Goal3_trend_analysis_by_year.csv"
pandas_trend_analysis_by_year.to_csv(local_path, index=False)

In [12]:
# 4. Determine how different types of transmissions (e.g., automatic, manual) influence fuel efficiency to guide transmission selection in vehicle design.
transmission_efficiency = df_clean.groupBy('TRANSMISSION').agg(
    round(avg('FUEL CONSUMPTION'), 2).alias('avg_fuel_consumption')
)

# Show the results to compare fuel efficiency by transmission type
transmission_efficiency.show(transmission_efficiency.count(), truncate=False)

pandas_transmission_efficiency = transmission_efficiency.toPandas()
local_path = "Goal4_transmission_efficiency.csv"
pandas_transmission_efficiency.to_csv(local_path, index=False)

+------------+--------------------+
|TRANSMISSION|avg_fuel_consumption|
+------------+--------------------+
|AS4         |11.72               |
|AS10        |13.95               |
|A9          |12.37               |
|AV10        |8.5                 |
|AM5         |6.13                |
|A6          |14.62               |
|AV1         |7.44                |
|M4          |8.4                 |
|M7          |12.91               |
|A8          |13.89               |
|AV          |8.23                |
|AM9         |11.1                |
|AS9         |11.89               |
|AS6         |12.37               |
|M6          |12.06               |
|AV8         |9.79                |
|AV6         |8.56                |
|AS7         |13.17               |
|A10         |14.77               |
|A4          |14.2                |
|AS8         |12.97               |
|M5          |10.78               |
|AS5         |12.76               |
|AV7         |9.0                 |
|A7          |13.65         

In [13]:
# 5: Evaluate the relationship between fuel consumption and emissions, providing insights into the ecological footprint of various vehicles.
correlation = df_clean.corr('FUEL CONSUMPTION', 'EMISSIONS')
print(f"Correlation between Fuel Consumption and Emissions: {correlation}")

Correlation between Fuel Consumption and Emissions: 0.9191005620020016


In [14]:
# 6. Analyze whether vehicles with more cylinders tend to consume more fuel.
selected_df = df_clean.select('ENGINE SIZE', 'CYLINDERS', 'COMB (L/100 km)')
fuel_consumption_by_cylinders = selected_df.groupBy('CYLINDERS').agg(
    round(avg('COMB (L/100 km)'), 2).alias('avg_fuel_consumption')
)
fuel_consumption_by_cylinders = fuel_consumption_by_cylinders.orderBy('avg_fuel_consumption', ascending=True)
fuel_consumption_by_cylinders.show()

pandas_fuel_consumption_by_cylinders = fuel_consumption_by_cylinders.toPandas()
local_path = "Goal6_fuel_consumption_by_cylinders.csv"
pandas_fuel_consumption_by_cylinders.to_csv(local_path, index=False)

+---------+--------------------+
|CYLINDERS|avg_fuel_consumption|
+---------+--------------------+
|        3|                7.12|
|        4|                8.61|
|        5|               10.12|
|        6|               11.18|
|        2|               11.18|
|        8|               14.15|
|       10|                15.4|
|       12|               16.33|
|       16|               23.08|
+---------+--------------------+



In [15]:
# 7. To analyze how different fuel types (e.g., gasoline, diesel, electric) affect fuel consumption and identify trends in the popularity and efficiency of different fuel sources.
selected_df = df_clean.select('FUEL', 'COMB (L/100 km)')
fuel_consumption_by_fuel_type = selected_df.groupBy('FUEL').agg(
    round(avg('COMB (L/100 km)'), 2).alias('avg_fuel_consumption')
)
fuel_consumption_by_fuel_type = fuel_consumption_by_fuel_type.orderBy('avg_fuel_consumption', ascending=True)
fuel_consumption_by_fuel_type.show()

pandas_fuel_consumption_by_fuel_type = fuel_consumption_by_fuel_type.toPandas()
local_path = "Goal5_fuel_consumption_by_fuel_type.csv"
pandas_fuel_consumption_by_fuel_type.to_csv(local_path, index=False)

+----+--------------------+
|FUEL|avg_fuel_consumption|
+----+--------------------+
|   D|                8.55|
|   X|               10.39|
|   Z|               11.26|
|   N|               16.59|
|   E|               16.78|
+----+--------------------+



In [16]:
# 8. Calculate the average fuel consumption (L/100 km) for vehicles with specific transmission types, grouped by vehicle class and ordered by the most fuel-efficient combination.
grouped_data = df.groupBy('TRANSMISSION', 'VEHICLE CLASS').agg(
    avg('FUEL CONSUMPTION').alias('avg_fuel_consumption')
)
ordered_data = grouped_data.orderBy('avg_fuel_consumption')
ordered_data.count()

405

In [17]:
# 9. Find the top 5 vehicle makes with the most significant change in fuel consumption over the years.
from pyspark.sql.window import Window
window_spec = Window.partitionBy('MAKE').orderBy('YEAR')
fuel_change = df_clean.withColumn('change', round(((df_clean['FUEL CONSUMPTION'] - lag(df_clean['FUEL CONSUMPTION']).over(window_spec)) / lag(df_clean['FUEL CONSUMPTION']).over(window_spec) * 100), 2))
top_makes_change = fuel_change.groupBy('MAKE').agg(round(sum('change'), 2).alias('sum_change')).orderBy('sum_change', ascending=False).limit(5)
top_makes_change.show()

+---------+----------+
|     MAKE|sum_change|
+---------+----------+
|CHEVROLET|   5513.99|
|     FORD|   5034.29|
|      GMC|   3731.19|
|   TOYOTA|   3259.56|
|    HONDA|   2706.06|
+---------+----------+



In [18]:
# 10. Identify the top 3 vehicle makes with the highest average emissions (EMISSIONS) for each year, considering the year-to-year change.
emissions_change_df = df_clean.withColumn('emissions_change', df_clean['EMISSIONS'] - lag(df_clean['EMISSIONS']).over(window_spec))
emissions_change_avg_df = emissions_change_df.groupBy('YEAR', 'MAKE').agg(round(avg('emissions_change'), 2).alias('avg_emissions_change'))
emissions_change_avg_df = emissions_change_avg_df.orderBy(['YEAR', desc('avg_emissions_change')])
emissions_change_avg_df.count()

891